In [15]:
import json
import statistics
import Levenshtein
import plyj.parser as plyj
import numpy as np
from scipy import stats
from tree_sitter import Language, Parser
from cliffs_delta import cliffs_delta as cd
import numpy as np
from scipy.stats import mannwhitneyu, wilcoxon

percentage = 20
file_id = f"gpt2/{percentage}/test_CodeGPT-small-java_checkpoint-epoch-5_victim.json"
# Path of the file containing perpexity and zlib scores
pz_path = "/mnt/hdd1/XXXX/LM_Memorization/" + file_id
# Path of the file containing 'id', 'input', 'gt', 'prediction', 'label', 'predicition_label' for each datapoint in a json format
input_path = f"/mnt/hdd1/XXXX/XXXX/Privacy-in-Code-Models/Classifier/classifier_save/PTM3/javaCorpus/CodeGPT-small-java/{percentage}/30/res_20_30.json"

In [16]:
# Load files
data_dict = {}
with open(input_path, "r") as file:
    for line_no, line in enumerate(file.readlines()):
        json_line = json.loads(line)
        if line_no in data_dict:
            print("Line number already exists:", line_no)
        data_dict[line_no] = json_line

with open(pz_path, "r") as file:
    for line_no, line in enumerate(file.readlines()):
        json_line = json.loads(line)

        # Check if the line_no exists in data_dict, and append perplexity and zlib values
        if line_no in data_dict:
            data_dict[line_no]["perplexity"] = json_line["perplexity"]
            data_dict[line_no]["zlib"] = json_line["zlib"]
            data_dict[line_no]["perplexity_compare"] = json_line["perplexity_compare"]
            data_dict[line_no]["zlib_compare"] = json_line["zlib_compare"]
        else:
            print("Line number not found:", line_no)


print ("Number of samples:", len(data_dict))
#print (f"The first sample: {data_dict[0]}")
print(f"Keys: {data_dict[0].keys()}")
data_dict_pred_label_0 = {k: v for k, v in data_dict.items() if v["predicition_label"] == 0}
data_dict_pred_label_1 = {k: v for k, v in data_dict.items() if v["predicition_label"] == 1}
print(f"Number of samples with predicition_label 0: {len(data_dict_pred_label_0)}")
print(f"Number of samples with predicition_label 1: {len(data_dict_pred_label_1)}")




Number of samples: 12980
Keys: dict_keys(['id', 'input', 'gt', 'prediction', 'label', 'predicition_label', 'perplexity', 'zlib', 'perplexity_compare', 'zlib_compare'])
Number of samples with predicition_label 0: 6401
Number of samples with predicition_label 1: 6579


In [17]:
def calculate_metrics(predictions, ground_truths):
    tp = sum(1 for p, gt in zip(predictions, ground_truths) if p == gt == 1)
    tn = sum(1 for p, gt in zip(predictions, ground_truths) if p == gt == 0)
    fp = sum(1 for p, gt in zip(predictions, ground_truths) if p == 1 and gt == 0)
    fn = sum(1 for p, gt in zip(predictions, ground_truths) if p == 0 and gt == 1)

    precision = tp / (tp + fp) if tp + fp > 0 else 0
    recall = tp / (tp + fn) if tp + fn > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if precision + recall > 0 else 0
    accuracy = (tp + tn) / (tp + tn + fp + fn) if tp + tn + fp + fn > 0 else 0
    tpr = tp / (tp + fn) if tp + fn > 0 else 0
    fpr = fp / (fp + tn) if fp + tn > 0 else 0

    return precision, recall, f1, accuracy, tpr, fpr

In [18]:
def inference():
    data = data_dict.values()
    # Calculate precision, recall, f1, accuracy for different thresholds
    for metric_name in ["perplexity", "zlib"]:
        if metric_name == "perplexity":
            values = sorted([entry[metric_name] for entry in data])
        elif metric_name == "zlib":
            values = sorted([entry[metric_name] for entry in data], reverse=True)

        thresholds = {f"{k}%": values[int(len(values) * k / 100)] for k in range(10, 70, 10)}
        ground_truths = [entry["label"] for entry in data]

        for threshold_name, threshold in thresholds.items():
            if metric_name == "perplexity":
                predictions = [1 if entry[metric_name] <= threshold else 0 for entry in data]
            elif metric_name == "zlib":
                predictions = [1 if entry[metric_name] >= threshold else 0 for entry in data]
            precision, recall, f1, accuracy, tpr, fpr = calculate_metrics(predictions, ground_truths)

            print(f"{metric_name} - {threshold_name}:")
            print(f"  Precision: {precision:.4f}")
            print(f"  Recall: {recall:.4f}")
            print(f"  F1: {f1:.4f}")
            print(f"  Accuracy: {accuracy:.4f}")
            print()
inference()

perplexity - 10%:
  Precision: 0.5466
  Recall: 0.1094
  F1: 0.1823
  Accuracy: 0.5093

perplexity - 20%:
  Precision: 0.5818
  Recall: 0.2328
  F1: 0.3326
  Accuracy: 0.5327

perplexity - 30%:
  Precision: 0.5884
  Recall: 0.3532
  F1: 0.4414
  Accuracy: 0.5531

perplexity - 40%:
  Precision: 0.5846
  Recall: 0.4678
  F1: 0.5197
  Accuracy: 0.5677

perplexity - 50%:
  Precision: 0.5763
  Recall: 0.5764
  F1: 0.5764
  Accuracy: 0.5763

perplexity - 60%:
  Precision: 0.5628
  Recall: 0.6755
  F1: 0.6140
  Accuracy: 0.5754

zlib - 10%:
  Precision: 0.2479
  Recall: 0.0496
  F1: 0.0827
  Accuracy: 0.4495

zlib - 20%:
  Precision: 0.3400
  Recall: 0.1361
  F1: 0.1943
  Accuracy: 0.4360

zlib - 30%:
  Precision: 0.3892
  Recall: 0.2336
  F1: 0.2920
  Accuracy: 0.4335

zlib - 40%:
  Precision: 0.4204
  Recall: 0.3364
  F1: 0.3737
  Accuracy: 0.4363

zlib - 50%:
  Precision: 0.4522
  Recall: 0.4522
  F1: 0.4522
  Accuracy: 0.4522

zlib - 60%:
  Precision: 0.4723
  Recall: 0.5669
  F1: 0.5153


In [19]:
def stat_result(title, data_list):
    # Calculate the statistics
    mean = statistics.mean(data_list)
    median = statistics.median(data_list)
    min_tokens = min(data_list)
    max_tokens = max(data_list)
    stdev = statistics.stdev(data_list) if len(data_list) > 1 else 0

    # Print the statistics
    print(title)
    print(f"- Number of samples: {len(data_list)}")
    print(f"- Mean: {mean}")
    print(f"- Median: {median}")
    print(f"- Minimum: {min_tokens}")
    print(f"- Maximum: {max_tokens}")
    print(f"- Standard deviation: {stdev}")
    print()

In [20]:
def cliffs_delta(x,y): # cohen_d
    number = abs(np.mean(x) - np.mean(y)) / np.sqrt((np.std(x, ddof=1) ** 2 + np.std(y, ddof=1) ** 2) / 2.0)
    size = ""
    if number < 0.2:
        size =  'negligible'
    elif number < 0.5:
        size =  'small'
    elif number < 0.8:
        size =  'medium'
    elif number < 1.3:
        size =  'large'
    else:
        size =  'very large'
    return number,size

def delta_alt(x, y):
    number = cd(x, y)
    return number

def stat_testing(list_1, list_2):

    # Perform a two-sample t-test
    result = stats.ranksums(list_1, list_2)
    delta_1 = cliffs_delta(list_1, list_2)
    delta_2 = delta_alt(list_1, list_2)
    
    stat, p_value = mannwhitneyu(list_1, list_2)
    rbc = 1 - (2 * stat) / (len(list_1) * len(list_2))
    
    

    print(f"T-statistic: {result.statistic}")
    print(f"P-value: {result.pvalue}")
    print(f"Cohens-Delta: {delta_1[0]} - {delta_1[1]}")
    print(f"Cliffs-Delta: {delta_2[0]} - {delta_2[1]}")
    print(f"Mann-Whitney U test Statistic: {stat:.2f}")
    print(f"Mann-Whitney U test P-value: {p_value:.2e}")
    print(f"Rank-biserial correlation: {rbc:.2f}")


In [21]:
# Stat 0: Perplexity and Zlib of the samples by pred_label
for metric in ["perplexity", "zlib"]:
    metric_list = [entry[metric] for entry in data_dict.values()]
    category_title = f"{metric} of the samples"
    stat_result(category_title + " (ALL)", metric_list)
    metric_list_pred_label_0 = [entry[metric] for entry in data_dict_pred_label_0.values()]
    metric_list_pred_label_1 = [entry[metric] for entry in data_dict_pred_label_1.values()]
    stat_result(category_title + " (Prediction Label = 1)", metric_list_pred_label_1)
    stat_result(category_title + " (Prediction Label = 0)", metric_list_pred_label_0)
    stat_testing(metric_list_pred_label_1, metric_list_pred_label_0)
    print()

perplexity of the samples (ALL)
- Number of samples: 12980
- Mean: 18.137249623480123
- Median: 4.944484233856201
- Minimum: 1.1290384531021118
- Maximum: 9417.759765625
- Standard deviation: 105.69412363862935

perplexity of the samples (Prediction Label = 1)
- Number of samples: 6579
- Mean: 8.756882474349613
- Median: 4.397513389587402
- Minimum: 1.3402657508850098
- Maximum: 1182.55859375
- Standard deviation: 26.192915778653763

perplexity of the samples (Prediction Label = 0)
- Number of samples: 6401
- Mean: 27.778467476023415
- Median: 5.769089698791504
- Minimum: 1.1290384531021118
- Maximum: 9417.759765625
- Standard deviation: 147.53430568275073

T-statistic: -23.63328694913563
P-value: 1.753494107935673e-123
Cohens-Delta: 0.17952706301948257 - negligible
Cliffs-Delta: -0.2395596770235993 - small
Mann-Whitney U test Statistic: 16011899.50
Mann-Whitney U test P-value: 1.75e-123
Rank-biserial correlation: 0.24

zlib of the samples (ALL)
- Number of samples: 12980
- Mean: 0.442

In [22]:
# Stat 0.2: Perplexity and Zlib of the samples by label
for metric in ["perplexity", "zlib"]:
    metric_list = [entry[metric] for entry in data_dict.values()]
    category_title = f"{metric} of the samples"
    stat_result(category_title + " (ALL)", metric_list)
    metric_list_label_0 = [entry[metric] for entry in data_dict.values() if entry["label"] == 0]
    metric_list_label_1 = [entry[metric] for entry in data_dict.values() if entry["label"] == 1]
    stat_result(category_title + " (Label = 1)", metric_list_label_1)
    stat_result(category_title + " (Label = 0)", metric_list_label_0)
    stat_testing(metric_list_label_1, metric_list_label_0)
    print()

perplexity of the samples (ALL)
- Number of samples: 12980
- Mean: 18.137249623480123
- Median: 4.944484233856201
- Minimum: 1.1290384531021118
- Maximum: 9417.759765625
- Standard deviation: 105.69412363862935

perplexity of the samples (Label = 1)
- Number of samples: 6490
- Mean: 9.190490091728687
- Median: 4.469345331192017
- Minimum: 1.3402657508850098
- Maximum: 1182.55859375
- Standard deviation: 29.606217568890383

perplexity of the samples (Label = 0)
- Number of samples: 6490
- Mean: 27.08400915523156
- Median: 5.591537714004517
- Minimum: 1.1290384531021118
- Maximum: 9417.759765625
- Standard deviation: 145.9711454657945

T-statistic: -20.86329377647766
P-value: 1.1544087371872662e-96
Cohens-Delta: 0.1698986067597524 - negligible
Cliffs-Delta: -0.21146165369977754 - small
Mann-Whitney U test Statistic: 16606657.00
Mann-Whitney U test P-value: 1.15e-96
Rank-biserial correlation: 0.21

zlib of the samples (ALL)
- Number of samples: 12980
- Mean: 0.44283757776552946
- Median: 

In [23]:
# Stat 1: The number of tokens in an example input
num_tokenized_inputs = [len(entry["input"].split()) for entry in data_dict.values()]
category_title = "Number of tokens in an example input"
stat_result(category_title + " (ALL)", num_tokenized_inputs)
num_tokenized_inputs_predicition_label_1 = [len(entry["input"].split()) for entry in data_dict.values() if entry["predicition_label"] == 1]
stat_result(category_title + " (Prediction Label = 1)", num_tokenized_inputs_predicition_label_1)
num_tokenized_inputs_predicition_label_0 = [len(entry["input"].split()) for entry in data_dict.values() if entry["predicition_label"] == 0]
stat_result(category_title + " (Prediction Label = 0)", num_tokenized_inputs_predicition_label_0)
stat_testing(num_tokenized_inputs_predicition_label_1, num_tokenized_inputs_predicition_label_0)


Number of tokens in an example input (ALL)
- Number of samples: 12980
- Mean: 305.9798151001541
- Median: 200.0
- Minimum: 5
- Maximum: 1199
- Standard deviation: 293.4212796680094

Number of tokens in an example input (Prediction Label = 1)
- Number of samples: 6579
- Mean: 326.07204742362063
- Median: 230
- Minimum: 5
- Maximum: 1198
- Standard deviation: 288.2441455088303

Number of tokens in an example input (Prediction Label = 0)
- Number of samples: 6401
- Mean: 285.3288548664271
- Median: 162
- Minimum: 5
- Maximum: 1199
- Standard deviation: 297.2593072450871

T-statistic: 13.545474644914854
P-value: 8.42674442460592e-42
Cohens-Delta: 0.13915669874076791 - negligible
Cliffs-Delta: 0.1373041988637064 - negligible
Mann-Whitney U test Statistic: 23947179.00
Mann-Whitney U test P-value: 8.42e-42
Rank-biserial correlation: -0.14


In [24]:
# Stat 2: The number of tokens in an example output
num_tokenized_outputs = [len(entry["prediction"].split()) for entry in data_dict.values()]
category_title = "Number of tokens in an example output"
stat_result(category_title + " (ALL)", num_tokenized_outputs)
num_tokenized_outputs_predicition_label_1 = [len(entry["prediction"].split()) for entry in data_dict.values() if entry["predicition_label"] == 1]
stat_result(category_title + " (Prediction Label = 1)", num_tokenized_outputs_predicition_label_1)
num_tokenized_outputs_predicition_label_0 = [len(entry["prediction"].split()) for entry in data_dict.values() if entry["predicition_label"] == 0]
stat_result(category_title + " (Prediction Label = 0)", num_tokenized_outputs_predicition_label_0)
stat_testing(num_tokenized_outputs_predicition_label_1, num_tokenized_outputs_predicition_label_0)

Number of tokens in an example output (ALL)
- Number of samples: 12980
- Mean: 5.873882896764253
- Median: 6.0
- Minimum: 0
- Maximum: 75
- Standard deviation: 3.431166849371259

Number of tokens in an example output (Prediction Label = 1)
- Number of samples: 6579
- Mean: 6.063231494148047
- Median: 6
- Minimum: 0
- Maximum: 68
- Standard deviation: 3.0440591473884657

Number of tokens in an example output (Prediction Label = 0)
- Number of samples: 6401
- Mean: 5.679268864239963
- Median: 5
- Minimum: 0
- Maximum: 75
- Standard deviation: 3.778403659663294

T-statistic: 9.104608083213428
P-value: 8.657845537803609e-20
Cohens-Delta: 0.11191195023362839 - negligible
Cliffs-Delta: 0.09228919263474825 - negligible
Mann-Whitney U test Statistic: 22999339.00
Mann-Whitney U test P-value: 4.32e-20
Rank-biserial correlation: -0.09


In [25]:
# Stat 3: The edit distance between the victim model output and the ground truth.
outputs = [entry["prediction"] for entry in data_dict.values()]
gts = [entry["gt"] for entry in data_dict.values()]
edit_distances = [Levenshtein.distance(model_output, ground_truth) for model_output, ground_truth in zip(outputs, gts)]
category_title = "Edit distance between the victim model output and the ground truth"
stat_result(category_title + " (ALL)", edit_distances)
outputs_predicition_label_1 = [entry["prediction"] for entry in data_dict.values() if entry["predicition_label"] == 1]
gts_predicition_label_1 = [entry["gt"] for entry in data_dict.values() if entry["predicition_label"] == 1]
outputs_predicition_label_0 = [entry["prediction"] for entry in data_dict.values() if entry["predicition_label"] == 0]
gts_predicition_label_0 = [entry["gt"] for entry in data_dict.values() if entry["predicition_label"] == 0]
edit_distances_predicition_label_1 = [Levenshtein.distance(model_output, ground_truth) for model_output, ground_truth in zip(outputs_predicition_label_1, gts_predicition_label_1)]
edit_distances_predicition_label_0 = [Levenshtein.distance(model_output, ground_truth) for model_output, ground_truth in zip(outputs_predicition_label_0, gts_predicition_label_0)]
stat_result(category_title + " (Prediction Label = 1)", edit_distances_predicition_label_1)
stat_result(category_title + " (Prediction Label = 0)", edit_distances_predicition_label_0)
stat_testing(edit_distances_predicition_label_1, edit_distances_predicition_label_0)

Edit distance between the victim model output and the ground truth (ALL)
- Number of samples: 12980
- Mean: 17.502927580893683
- Median: 13.0
- Minimum: 1
- Maximum: 1722
- Standard deviation: 23.76555964331625

Edit distance between the victim model output and the ground truth (Prediction Label = 1)
- Number of samples: 6579
- Mean: 16.209454324365407
- Median: 11
- Minimum: 1
- Maximum: 186
- Standard deviation: 17.911344507791

Edit distance between the victim model output and the ground truth (Prediction Label = 0)
- Number of samples: 6401
- Mean: 18.832369942196532
- Median: 15
- Minimum: 1
- Maximum: 1722
- Standard deviation: 28.498665355120078

T-statistic: -9.648956728735286
P-value: 4.9658339826938e-22
Cohens-Delta: 0.11020117935881027 - negligible
Cliffs-Delta: -0.09780700257756789 - negligible
Mann-Whitney U test Statistic: 18996656.50
Mann-Whitney U test P-value: 1.29e-22
Rank-biserial correlation: 0.10


In [26]:
# Stat 4: The number of variable names in the example (input + gt)
# You need to install tree-sitter and tree-sitter-java and set the path to the language.so file
path = '/mnt/hdd1/XXXX/LM_Memorization/python_parser/parser_folder/my-languages.so'
parser = Parser()
parser.set_language(Language(path, "java"))

def num_var(code_snippet):
    tree = parser.parse(bytes(code_snippet, "utf8"))
    def count_variable_declarations(node):
        variable_count = 0
        if node.type == 'variable_declarator':
            variable_count += 1

        for child in node.children:
            variable_count += count_variable_declarations(child)

        return variable_count

    root_node = tree.root_node
    variable_count = count_variable_declarations(root_node)
    return variable_count

num_var_all = [num_var(entry["input"] + entry["gt"]) for entry in data_dict.values()]
num_var_predicition_label_1 = [num_var(entry["input"] + entry["gt"]) for entry in data_dict.values() if entry["predicition_label"] == 1]
num_var_predicition_label_0 = [num_var(entry["input"] + entry["gt"]) for entry in data_dict.values() if entry["predicition_label"] == 0]
category_title = "Number of variable names in the example (input + gt)"
stat_result(category_title + " (ALL)", num_var_all)
stat_result(category_title + " (Prediction Label = 1)", num_var_predicition_label_1)
stat_result(category_title + " (Prediction Label = 0)", num_var_predicition_label_0)
stat_testing(num_var_predicition_label_1, num_var_predicition_label_0)


Number of variable names in the example (input + gt) (ALL)
- Number of samples: 12980
- Mean: 6.244144838212635
- Median: 3.0
- Minimum: 0
- Maximum: 205
- Standard deviation: 8.670587176147551

Number of variable names in the example (input + gt) (Prediction Label = 1)
- Number of samples: 6579
- Mean: 6.664994680042559
- Median: 4
- Minimum: 0
- Maximum: 154
- Standard deviation: 8.629652968043496

Number of variable names in the example (input + gt) (Prediction Label = 0)
- Number of samples: 6401
- Mean: 5.811591938759569
- Median: 3
- Minimum: 0
- Maximum: 205
- Standard deviation: 8.691921355831518

T-statistic: 8.3280420910604
P-value: 8.219029443623229e-17
Cohens-Delta: 0.09853575791135999 - negligible
Cliffs-Delta: 0.08441750306959894 - negligible
Mann-Whitney U test Statistic: 22833592.00
Mann-Whitney U test P-value: 4.93e-17
Rank-biserial correlation: -0.08


In [28]:
from sklearn.metrics import roc_auc_score
# Stat 5: Power Error and AUC of K = 50% prediction based on Perplexity, Compare Perplexity and Compare Zlib
def inference_2():
    data = data_dict.values()
    # Calculate precision, recall, f1, accuracy for different thresholds
    for metric_name in ["perplexity", "perplexity_compare","zlib_compare"]:
        
        values = sorted([entry[metric_name] for entry in data])
        thresholds = {f"{k}%": values[int(len(values) * k / 100)] for k in range(10, 71, 10)}
        ground_truths = [entry["label"] for entry in data]

        for threshold_name, threshold in thresholds.items():
            predictions = [1 if entry[metric_name] <= threshold else 0 for entry in data]
            precision, recall, f1, accuracy, tpr, fpr = calculate_metrics(predictions, ground_truths)
            auc = roc_auc_score(ground_truths, predictions)

            print(f"{metric_name} - {threshold_name}:")
            print(f"  Precision: {precision:.4f}")
            print(f"  Recall: {recall:.4f}")
            print(f"  F1: {f1:.4f}")
            print(f"  Accuracy: {accuracy:.4f}")
            print(f"  Power(TPR): {tpr:.4f}")
            print(f"  Error(FPR): {fpr:.4f}")
            print(f"  AUC: {auc:.4f}")
            print()
inference_2()

perplexity - 10%:
  Precision: 0.5466
  Recall: 0.1094
  F1: 0.1823
  Accuracy: 0.5093
  Power(TPR): 0.1094
  Error(FPR): 0.0908
  AUC: 0.5093

perplexity - 20%:
  Precision: 0.5818
  Recall: 0.2328
  F1: 0.3326
  Accuracy: 0.5327
  Power(TPR): 0.2328
  Error(FPR): 0.1673
  AUC: 0.5327

perplexity - 30%:
  Precision: 0.5884
  Recall: 0.3532
  F1: 0.4414
  Accuracy: 0.5531
  Power(TPR): 0.3532
  Error(FPR): 0.2470
  AUC: 0.5531

perplexity - 40%:
  Precision: 0.5846
  Recall: 0.4678
  F1: 0.5197
  Accuracy: 0.5677
  Power(TPR): 0.4678
  Error(FPR): 0.3324
  AUC: 0.5677

perplexity - 50%:
  Precision: 0.5763
  Recall: 0.5764
  F1: 0.5764
  Accuracy: 0.5763
  Power(TPR): 0.5764
  Error(FPR): 0.4237
  AUC: 0.5763

perplexity - 60%:
  Precision: 0.5628
  Recall: 0.6755
  F1: 0.6140
  Accuracy: 0.5754
  Power(TPR): 0.6755
  Error(FPR): 0.5247
  AUC: 0.5754

perplexity - 70%:
  Precision: 0.5563
  Recall: 0.7789
  F1: 0.6490
  Accuracy: 0.5788
  Power(TPR): 0.7789
  Error(FPR): 0.6213
  AUC: 

In [29]:
import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score
from tabulate import tabulate
import zlib


def inference_2_csv():
    data = data_dict.values()
    results = []

    for metric_name in ["perplexity", "perplexity_compare", "zlib_compare"]:
        values = sorted([entry[metric_name] for entry in data])
        thresholds = {f"{k}%": values[int(len(values) * k / 100)] for k in range(10, 71, 10)}
        ground_truths = [entry["label"] for entry in data]

        for threshold_name, threshold in thresholds.items():
            predictions = [1 if entry[metric_name] <= threshold else 0 for entry in data]
            precision, recall, f1, accuracy, tpr, fpr = calculate_metrics(predictions, ground_truths)
            auc = roc_auc_score(ground_truths, predictions)

            results.append({
                "Metric": metric_name,
                "Threshold": threshold_name,
                "Precision": precision,
                "Recall": recall,
                "F1": f1,
                "Accuracy": accuracy,
                "Power (TPR)": tpr,
                "Error (FPR)": fpr,
                "AUC": auc
            })

    df = pd.DataFrame(results)
    print(tabulate(df, headers='keys', tablefmt='pretty'))
    df.to_csv('power.csv', index=False)

inference_2_csv()


+----+--------------------+-----------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+
|    |       Metric       | Threshold |      Precision      |       Recall        |         F1          |      Accuracy       |     Power (TPR)     |     Error (FPR)     |         AUC         |
+----+--------------------+-----------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+
| 0  |     perplexity     |    10%    | 0.5465742879137798  | 0.10939907550077041 | 0.1823083836179227  | 0.5093220338983051  | 0.10939907550077041 | 0.09075500770416024 |  0.509322033898305  |
| 1  |     perplexity     |    20%    | 0.5818251829033501  | 0.23281972265023113 | 0.3325630020908991  | 0.5327426810477658  | 0.23281972265023113 | 0.16733436055469955 | 0.5327426810477659  |
| 2  |     perplexity     |   